In [25]:
bucket = 'testsmdatanatey'
prefix = 'sagemaker/DEMO-xghboost-churn'

#define IAM role

import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os 
import io
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
import sagemaker
from sagemaker.predictor import csv_serializer

In [27]:
!wget http://dataminingconsultant.com/DKD2e_data_sets.zip
!unzip -o DKD2e_data_sets.zip

--2020-05-16 23:49:24--  http://dataminingconsultant.com/DKD2e_data_sets.zip
Resolving dataminingconsultant.com (dataminingconsultant.com)... 160.153.91.162
Connecting to dataminingconsultant.com (dataminingconsultant.com)|160.153.91.162|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1003616 (980K) [application/zip]
Saving to: ‘DKD2e_data_sets.zip.4’

DKD2e_data_sets.zip 100%[===================>] 980.09K  5.86MB/s    in 0.2s    

2020-05-16 23:49:24 (5.86 MB/s) - ‘DKD2e_data_sets.zip.4’ saved [1003616/1003616]

Archive:  DKD2e_data_sets.zip
 extracting: Data sets/adult.zip     
  inflating: Data sets/cars.txt      
  inflating: Data sets/cars2.txt     
  inflating: Data sets/cereals.CSV   
  inflating: Data sets/churn.txt     
  inflating: Data sets/ClassifyRisk  
  inflating: Data sets/ClassifyRisk - Missing.txt  
 extracting: Data sets/DKD2e data sets.zip  
  inflating: Data sets/nn1.txt       


In [28]:
churn = pd.read_csv('./Data sets/churn.txt')
pd.set_option('display.max_columns', 500)
churn.head()

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False.
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False.
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False.
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False.
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False.


In [29]:
churn.columns

Index(['State', 'Account Length', 'Area Code', 'Phone', 'Int'l Plan',
       'VMail Plan', 'VMail Message', 'Day Mins', 'Day Calls', 'Day Charge',
       'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls',
       'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge',
       'CustServ Calls', 'Churn?'],
      dtype='object')

In [30]:
churn = churn.drop('Phone', axis = 1)
churn['Area Code'] = churn['Area Code'].astype(object)
churn.select_dtypes(include = 'object').head()

,State,Area Code,Int'l Plan,VMail Plan,Churn?
0,KS,415,no,yes,False.
1,OH,415,no,yes,False.
2,NJ,415,no,no,False.
3,OH,408,yes,no,False.
4,OK,415,yes,no,False.


In [31]:
for column in churn.select_dtypes(include = ['object']).columns:
    if column != 'Churn?':
        display(pd.crosstab(index = churn[column], columns = churn['Churn?'], normalize='columns'))

Churn?,False.,True.
State,,
AK,0.017193,0.006211
AL,0.025263,0.016563
AR,0.015439,0.022774
AZ,0.021053,0.008282
CA,0.008772,0.018634
CO,0.020000,0.018634
CT,0.021754,0.024845
DC,0.017193,0.010352
DE,0.018246,0.018634


Churn?,False.,True.
Area Code,,
408,0.251228,0.252588
415,0.497895,0.488613
510,0.250877,0.258799


Churn?,False.,True.
Int'l Plan,,
no,0.934737,0.716356
yes,0.065263,0.283644


Churn?,False.,True.
VMail Plan,,
no,0.704561,0.834369
yes,0.295439,0.165631


In [32]:
model_data= pd.get_dummies(churn, drop_first=True)
model_data.head()

,Account Length,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,State_AL,State_AR,State_AZ,State_CA,State_CO,State_CT,State_DC,State_DE,State_FL,State_GA,State_HI,State_IA,State_ID,State_IL,State_IN,State_KS,State_KY,State_LA,State_MA,State_MD,State_ME,State_MI,State_MN,State_MO,State_MS,State_MT,State_NC,State_ND,State_NE,State_NH,State_NJ,State_NM,State_NV,State_NY,State_OH,State_OK,State_OR,State_PA,State_RI,State_SC,State_SD,State_TN,State_TX,State_UT,State_VA,State_VT,State_WA,State_WI,State_WV,State_WY,Area Code_415,Area Code_510,Int'l Plan_yes,VMail Plan_yes,Churn?_True.
0,128,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1,107,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
2,137,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,84,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,75,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0


Lets move the label 'churn?true.' column to be the first column in our dataframe.

In [33]:
model_data = pd.concat([model_data['Churn?_True.'], model_data.drop(['Churn?_True.'], axis = 1)], axis = 1)
model_data.head()

,Churn?_True.,Account Length,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,State_AL,State_AR,State_AZ,State_CA,State_CO,State_CT,State_DC,State_DE,State_FL,State_GA,State_HI,State_IA,State_ID,State_IL,State_IN,State_KS,State_KY,State_LA,State_MA,State_MD,State_ME,State_MI,State_MN,State_MO,State_MS,State_MT,State_NC,State_ND,State_NE,State_NH,State_NJ,State_NM,State_NV,State_NY,State_OH,State_OK,State_OR,State_PA,State_RI,State_SC,State_SD,State_TN,State_TX,State_UT,State_VA,State_VT,State_WA,State_WI,State_WV,State_WY,Area Code_415,Area Code_510,Int'l Plan_yes,VMail Plan_yes
0,0,128,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,0,107,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
2,0,137,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,0,84,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,75,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0


In [65]:
train_data,validation_data,test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7*len(model_data)), int(0.9 * len(model_data))])

In [35]:
train_data.to_csv('train.csv', header = False, index = False)
validation_data.to_csv('validation.csv', header=False, index = False)

Now we will upload these files to S3

In [37]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix,'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix,'validation/validation.csv')).upload_file('validation.csv')

**Lets Train**

In [38]:
region_name = boto3.Session(region_name= 'us-west-1').region_name
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(region_name, 'xgboost')
print('container name: {} and region: {}'.format(container, region_name))

	get_image_uri(region, 'xgboost', '0.90-1').


container name: 632365934929.dkr.ecr.us-west-1.amazonaws.com/xgboost:1 and region: us-west-1


then, because we are training with the csv file format,we will create s3_inputs that our training function can use as a pointer to s3.

In [39]:
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type = 'csv')
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/{}/validation'.format(bucket, prefix), content_type = 'csv')

In [40]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(container,
                                   role,
                                   train_instance_count=1,
                                   train_instance_type='ml.m4.xlarge', 
                                   output_path='s3://{}/{}/output'.format(bucket, prefix),
                                   sagemaker_session=sess)

xgb.set_hyperparameters(max_depth=5, 
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

xgb.fit({'train':s3_input_train, 'validation':s3_input_validation})

2020-05-16 23:52:03 Starting - Starting the training job...
2020-05-16 23:52:05 Starting - Launching requested ML instances......
2020-05-16 23:53:07 Starting - Preparing the instances for training......
2020-05-16 23:54:15 Downloading - Downloading input data...
2020-05-16 23:55:00 Training - Training image download completed. Training in progress..Arguments: train
[2020-05-16:23:55:00:INFO] Running standalone xgboost training.
[2020-05-16:23:55:00:INFO] File size need to be processed in the node: 0.5mb. Available memory size in the node: 8493.95mb
[2020-05-16:23:55:00:INFO] Determined delimiter of CSV input is ','
[23:55:00] S3DistributionType set as FullyReplicated
[23:55:00] 2333x69 matrix with 160977 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2020-05-16:23:55:00:INFO] Determined delimiter of CSV input is ','
[23:55:00] S3DistributionType set as FullyReplicated
[23:55:00] 666x69 matrix with 45954 entries loaded from /opt/ml/input/data/valida


2020-05-16 23:55:12 Uploading - Uploading generated training model
2020-05-16 23:55:12 Completed - Training job completed
Training seconds: 57
Billable seconds: 57


In [43]:
xgb.predictor = xgb.deploy(initial_instance_count =1,instance_type ='ml.m4.xlarge')

---------------!

**Evaluate**

Now that we have a hosted endpoint running,we can make real-time prediction from our model easyly,by making an http POST request.But, first we need to setup serializers and deserializers for passing our test_data NumPy arrays to the model behind the endpoint.  

In [46]:
xgb.predictor.content_type = 'text/csv'
xgb.predictor.serializer = csv_serializer
xgb.predictor.deserializer = None

In [66]:
test_data.shape

(334, 70)

In [67]:
test_data.head(1)

,Churn?_True.,Account Length,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,State_AL,State_AR,State_AZ,State_CA,State_CO,State_CT,State_DC,State_DE,State_FL,State_GA,State_HI,State_IA,State_ID,State_IL,State_IN,State_KS,State_KY,State_LA,State_MA,State_MD,State_ME,State_MI,State_MN,State_MO,State_MS,State_MT,State_NC,State_ND,State_NE,State_NH,State_NJ,State_NM,State_NV,State_NY,State_OH,State_OK,State_OR,State_PA,State_RI,State_SC,State_SD,State_TN,State_TX,State_UT,State_VA,State_VT,State_WA,State_WI,State_WV,State_WY,Area Code_415,Area Code_510,Int'l Plan_yes,VMail Plan_yes
2750,0,186,0,137.8,97,23.43,187.7,118,15.95,146.4,85,6.59,8.7,6,2.35,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


the test_data needs only the data without the out come variable 'Churn?_True.'.

In [68]:
test_data_1 = test_data.drop(['Churn?_True.'], axis = 1)

In [69]:
test_data_1.head(1).values

array([[186.  ,   0.  , 137.8 ,  97.  ,  23.43, 187.7 , 118.  ,  15.95,
        146.4 ,  85.  ,   6.59,   8.7 ,   6.  ,   2.35,   1.  ,   0.  ,
          0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   1.  ,   0.  ,
          0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  ,   0.  ,   1.  ,   0.  ,   0.  ]])

Therefore to see the probability of the customer with the attributes values in the first row of the test_data will churn lets run the following code.

In [88]:
xgb.predictor.predict(test_data_1.head(1).values)

b'0.0155654372647'

The single row evaluation in the above cell is interpreted as the probability that the customer in the first row will churn is 0.01556554372647.from the output cell 48 we can see that the customer did not churn with the value of 0 and the predicted value has shown that it is very unlikely that this customer will churn with a probability to chur very close to 0.

lets see the prediction result for all the rows in the test data to to evaluate our model performance.

In [71]:
# np.array_split(test_data.values, int(test_data.shape[0]/float(500) + 1))
test_data_1.values

array([[186. ,   0. , 137.8, ...,   1. ,   0. ,   0. ],
       [132. ,  25. , 113.2, ...,   0. ,   0. ,   1. ],
       [112. ,  17. , 183.2, ...,   0. ,   0. ,   1. ],
       ...,
       [ 86. ,   0. , 166.2, ...,   0. ,   0. ,   0. ],
       [ 36. ,  43. ,  29.9, ...,   0. ,   0. ,   1. ],
       [123. ,   0. , 163.1, ...,   0. ,   0. ,   0. ]])

In [72]:
prob_arr = []
for row in test_data_1.values:
    prob_arr.append(float(xgb.predictor.predict(row).decode('utf-8')))

Lets see the Confusion Matrix of the actual values versus Model prediction values.

In [76]:
pd.crosstab(test_data['Churn?_True.'],np.round(prob_arr))

col_0,0.0,1.0
Churn?_True.,,
0,282,4
1,9,39


Based on the result of our Confusion Matrix;

In [77]:
precision = (282 + 39)/(282 + 39 + 9 + 4)

In [78]:
precision

0.9610778443113772

In [80]:
senitivity = 39/(9 + 39)
specificity = 282/(282 + 4)
print('senitivity and specificity are respectively {}, and {}'.format(senitivity, specificity))

senitivity and specificity are respectively 0.8125, and 0.986013986013986


Both the specificity and sensitivity performances are strong which is an indication that our model is performing very well.

In [93]:
transformer = xgb.transformer(instance_count=1, 
                              instance_type='ml.m4.xlarge', 
                              output_path='s3://testsmdatanatey/batchoutput')
                                                

transformer.transform('s3://testsmdatanatey/sagemaker/DEMO-xghboost-churn/train', data_type='S3Prefix', content_type='text/csv', split_type='Line')


In [85]:
s3_input_train.config

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix',
   'S3Uri': 's3://testsmdatanatey/sagemaker/DEMO-xghboost-churn/train',
   'S3DataDistributionType': 'FullyReplicated'}},
 'ContentType': 'csv'}